In [1]:
%load_ext autoreload
%autoreload 2

# Symmetry of a molecule

In [2]:
import numpy as np
np.set_printoptions(linewidth=150)
from principia_materia.translation_group import Cluster
from principia_materia.representation import ClusterRep, DispClusterRep

## Symmetry of $CO_2$ molecule

### Representation of atoms in a molecule

In this example, we analyze the symmetry of a $CO_2$ molecule. 
Since the 3 atoms of the molecule are positioned along a line, the molecule has rotational symmetry along this axis. 
We can simplify the problem by studying the displacements along the rotational axis (referred to as $x$) and perpendicular to it ($y$), instead of all 3 dimensions.

For this 1 dimensional molecule, there is an inversion centered at the carbon atom, thus we can use point group $C_i$ for the symmetry analysis of this molecule.

The representation of the molecule atoms is straightforward since each species can only map into each other under point operations. That is the carbon atom has its own odd mode and the 2 oxygen atoms linearly combine into an odd and an even mode.

![co2](../translation_group/illustrations/molecule/co2.pdf)

First, we construct a _Cluster_ object with the atoms, and only $p_x$ and $p_y$ orbitals:

In [3]:
atoms = {
    "C": np.array([
        [0.0, 0.0, 0.0]
    ]),
    "O": np.array([
        [ 1.0, 0.0, 0.0],
        [-1.0, 0.0, 0.0]
    ])
}
cluster = Cluster(atoms=atoms, orbitals="p_x,p_y")

Using the _ClusterRep_ class, we can find that the representation of the molecule atoms decomposes into:
$
\Gamma = 2 A \oplus B
$


In [4]:
cluster_rep = ClusterRep(cluster=cluster, pg="Ci")
cluster_rep.irrep_counter

OrderedDict([('A', 2), ('B', 1)])

In [5]:
cluster_rep.construct_irreducible_basis()
cluster_rep.irrvec_dict

OrderedDict([(('A', 0), array([[1., 0., 0.]])),
             (('A', 1), array([[0.        , 0.70710678, 0.70710678]])),
             (('B', 0), array([[ 0.        ,  0.70710678, -0.70710678]]))])

And the basis of irreducible representations can also be computed easily:
$$
\vec{v}_{C, odd} = \vec{v}_{A} = \left[ 1, 0, 0 \right] ^T \\
\vec{v}_{O, odd} = \vec{v}_{A^{'}} = \frac{1}{\sqrt{2}}\left[ 0, 1, 1 \right] ^T \\
\vec{v}_{O, even} = \vec{v}_{B} = \frac{1}{\sqrt{2}}\left[ 0, 1, -1 \right] ^T \\
$$

### Displacements of a molecule

The symmetry analysis of the displacements of the molecule can be easily performed with _DispClusterRep_ class
to obtain the irreducible representations:
$$
\Gamma = 2 A \oplus 4 B
$$

In [6]:
disp_cluster_rep = DispClusterRep(cluster=cluster, pg="Ci")
disp_cluster_rep.irrep_counter

OrderedDict([('A', 2), ('B', 4)])

In [7]:
disp_cluster_rep.construct_irreducible_basis()
disp_cluster_rep.irrvec_dict

OrderedDict([(('A', 0),
              array([[ 0.        ,  0.        ,  0.70710678,  0.        , -0.70710678,  0.        ]])),
             (('A', 1),
              array([[ 0.        ,  0.        ,  0.        ,  0.70710678,  0.        , -0.70710678]])),
             (('B', 0), array([[1., 0., 0., 0., 0., 0.]])),
             (('B', 1), array([[0., 1., 0., 0., 0., 0.]])),
             (('B', 2),
              array([[0.        , 0.        , 0.70710678, 0.        , 0.70710678, 0.        ]])),
             (('B', 3),
              array([[0.        , 0.        , 0.        , 0.70710678, 0.        , 0.70710678]]))])

We can find the basis vectors of the irreducible representations:
$$
\vec{v}_{C, x, odd} = \vec{v}_{B} = \left[ 1, 0, 0, 0, 0, 0 \right] ^T \\
\vec{v}_{C, y, odd} = \vec{v}_{B'} = \left[ 0, 1, 0, 0, 0, 0 \right] ^T \\
%
\vec{v}_{O, x, odd} = \vec{v}_{B''} = \frac{1}{\sqrt{2}}\left[ 0, 0, 1, 0, 1, 0 \right] ^T \\
\vec{v}_{O, y, odd} = \vec{v}_{B'''} = \frac{1}{\sqrt{2}}\left[ 0, 0, 0, 1, 0, 1 \right] ^T \\
\vec{v}_{O, x, even} = \vec{v}_{A} = \frac{1}{\sqrt{2}}\left[ 0, 0, 1, 0, -1, 0 \right] ^T \\
\vec{v}_{O, y, even} = \vec{v}_{A'} = \frac{1}{\sqrt{2}}\left[ 0, 0, 0, 1, 0, -1 \right] ^T \\
$$

# Symmetry analysis for phonons

Here we demonstrate the symmetry analysis for phonons at a given $\textbf{q}$-point in a crystal, which can break down into several steps: 
1. Find the irreducible representaions of the nuclei displacements at said $\textbf{q}$-point;
2. Using the direct product or symmetric direct product to find the irreducible derivatives at second order;
3. Compute the coefficients for the irreducible derivatives in the dynamic matrix.



## Symmetry of Displacements of a Crystal

In [8]:
from principia_materia import Fraction
from principia_materia.io_interface import parse_array
from principia_materia.translation_group import CrystalFTG
from principia_materia.representation import CharmBlochRep

In [9]:
vec =  np.array([
    [0.0, 0.5, 0.5],
    [0.5, 0.0, 0.5],
    [0.5, 0.5, 0.0],
])
atoms = {
    "Na": np.array([
        [0.0, 0.0, 0.0]
    ]),
    "Cl": np.array([
        [0.5, 0.5, 0.5],
    ])
}
orbitals = "p"

In [10]:
structure = CrystalFTG(vec=vec, atoms=atoms, orbitals=orbitals)

In [11]:
cbr = CharmBlochRep(
    structure=structure,
    pg="Oh",
    qpoint=np.array(parse_array("0 0 0", dtype=Fraction)),
)

In [12]:
cbr.irrep_counter

OrderedDict([('T1u', 2)])

In [13]:
cbr.construct_irreducible_basis()
cbr.orthogonalize_shift_modes(remove=True)
print(cbr.irrep_counter)
for key, val in cbr.irrvec_dict.items():
    print(key)
    print(np.round(val, 6))

OrderedDict([('T1u', 1)])
('T1u', 0)
[[ 0.707107+0.j  0.      +0.j  0.      +0.j -0.707107+0.j  0.      +0.j  0.      +0.j]
 [ 0.      +0.j  0.707107+0.j  0.      +0.j  0.      +0.j -0.707107+0.j  0.      +0.j]
 [ 0.      +0.j  0.      +0.j  0.707107+0.j  0.      +0.j  0.      +0.j -0.707107+0.j]]


## Direct product and symmetric direct product of irreducible representations

In [14]:
from principia_materia.representation import DirectProduct, SymmetricDirectProduct

In [15]:
dp = DirectProduct(pg="Oh", irrep_inputs=["T1u", "T1u"])

In [16]:
dp.irrep_counter

OrderedDict([('A1g', 1), ('Eg', 1), ('T1g', 1), ('T2g', 1)])

In [17]:
dp.construct_irreducible_basis()

In [18]:
dp.irrvec_dict[("A1g", 0)]

array([[0.57735027, 0.        , 0.        , 0.        , 0.57735027, 0.        , 0.        , 0.        , 0.57735027]])

In [19]:
dp.basis

[(('T1u', 0), ('T1u', 0)),
 (('T1u', 0), ('T1u', 1)),
 (('T1u', 0), ('T1u', 2)),
 (('T1u', 1), ('T1u', 0)),
 (('T1u', 1), ('T1u', 1)),
 (('T1u', 1), ('T1u', 2)),
 (('T1u', 2), ('T1u', 0)),
 (('T1u', 2), ('T1u', 1)),
 (('T1u', 2), ('T1u', 2))]

In [20]:
sdp = SymmetricDirectProduct(pg="Oh", irrep_inputs=["T1u", "T1u"])

In [21]:
sdp.irrep_counter

OrderedDict([('A1g', 1), ('Eg', 1), ('T2g', 1)])

In [22]:
sdp.construct_irreducible_basis()

In [23]:
sdp.irrvec_dict[("A1g", 0)]

array([[0.57735027, 0.        , 0.        , 0.57735027, 0.        , 0.57735027]])

In [24]:
sdp.basis

[(('T1u', 0), ('T1u', 0)),
 (('T1u', 0), ('T1u', 1)),
 (('T1u', 0), ('T1u', 2)),
 (('T1u', 1), ('T1u', 1)),
 (('T1u', 1), ('T1u', 2)),
 (('T1u', 2), ('T1u', 2))]

In [25]:
sdp.get_integer_irrvec(("A1g", 0))

array([[1., 0., 0., 1., 0., 1.]])